In [142]:
from bs4 import BeautifulSoup #parser - can be made to crawl through while True + code as shown below
import requests #easilly send http requests to retrieve 
import concurrent.futures # Python build-in multi-threading library
import time

In [143]:
url = "https://boston.craigslist.org/search/ofc" #page we want to scrape and start crawling from
max_threads = 30

In [144]:
def get_soup(url): # equivalent to download_url - retrieves the html code of 'url'
    response = requests.get(url)
    data = response.text
    return BeautifulSoup(data, 'html.parser')

In [145]:
def parse_descriptions_url(jobs): # retrieves all descriptions urls within a given page
    links_list = []
    for job in jobs:
        links_list.append(job.find('a',{'class':'result-title'}).get('href'))
        
    return links_list

**Below function is where parallel computing will happen !**

In [146]:
def download_descriptions(links_list:list) -> list: # downloads all job descriptions in a given page and puts them in the list 'l'
    threads = min(max_threads, len(links_list))

    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        description_soups = executor.map(get_soup, links_list)
    return list(map(lambda x : x.find('section', {'id':'postingbody'}).text, description_soups))

# Asynchronous threads = might cause misatribution of descriptions as we don't know the order in which descriptions are added in descriptions_list

In [147]:
def main(url):

    while True:
        
        soup = get_soup(url)
        jobs = soup.find_all('div', {'class':'result-info'})

        links_list = parse_descriptions_url(jobs)
        descriptions_list = download_descriptions(links_list)
        
        i = 0

        for job in jobs: #scrapes job offers in from page(url) ---- should define function to do this step in multi aswell could help above problem

            title = job.find('a',{'class':'result-title'}).text
            location_tag = job.find('span', {'class':'result-hood'})
            location = location_tag.text[2:-1].strip() if location_tag else job.find('span', {'class':'nearby'}).text
            date = job.find('time', {'class':'result-date'}).text
            link = job.find('a',{'class':'result-title'}).get('href')


            print(f'Job title : {title}\nLocation : {location}\nDate : {date}\nLink : {link}\nDescription : {descriptions_list[i]}\n\n\n---\n\n\n')

            i += 1
        

        next_page_tag = get_soup(url).find('a', {'class':'button next'}) # next page div

        if next_page_tag.get('href'): # if there is a link, update url to crawl next page
            url = 'https://boston.craigslist.org' + next_page_tag.get('href')
        else:
            break

In [148]:
main(url)

Job title : TAX PREPARERS, OFFICE STAFF&WAVERS
Location : Lynn north shore
Date : Dec 26
Link : https://boston.craigslist.org/nos/ofc/d/lynn-tax-preparers-office-staffwavers/7425167380.html
Description : 

QR Code Link to This Post


Liberty Tax Service in Lynn is now hiring Tax Preparers, Office Support Staff, Wavers, and Marketers for tax season. 
Interested candidates will have strong computer, interpersonal skills, customer service skills, and the ability to work in a busy, fast paced environment. 

Flexible hours and bonuses available. Must be able to attend tax class training on January 3th-6th 10:00am to 4:00pm and January 7th 3pm to 9pm.

Wavers and Marketers must have a great personality with the ability to make people smile a must. Waver and marketing positions are part time only. 

Interviews will be conducted in person on December 27th thru December 30th. Please call 781 599-4110 for an interview. Please do not send emails.    


---



Job title : PT Jack of all trades
Loc

# F***ING AMAZING PERFORMANCE GAINs !!

Didn't expect it to work on the first try lmaooo

Seems super generalizable